In [ ]:
import utils
import tensorflow as tf
import time
import numpy as np
import math

preprocessed_data_path = './preprocessed_data/'
pred_path = './result/'

In [ ]:
x_train_d, y_train_d, w_train_d, e_train_d, x_test_d, id_test_d = utils.load_preprocessed_pd_data(preprocessed_data_path)

In [ ]:
print(x_train_d, '\n',
      y_train_d, '\n',
      w_train_d, '\n',
      e_train_d, '\n',
      x_test_d, '\n',
      id_test_d)

In [ ]:
epochs = 40
learning_rate = 0.001
n_unit = [48, 24, 12, 6, 3]
keep_prob_ = 1.0
batch_size = 128
display_step = 100
save_path = './checkpoints/'
log_path = './log/'

In [ ]:
list(x_train_d.shape)[1]

In [ ]:
feature_num = list(x_train_d.shape)[1]

inputs = tf.placeholder(tf.float64, [None, feature_num], name='inputs')
labels = tf.placeholder(tf.float64, None, name='labels')
weights = tf.placeholder(tf.float64, None, name='loss_weights')
is_train = tf.placeholder(tf.bool, name='is_train')
keep_prob = tf.placeholder(tf.float64, None, name='keep_prob')

def fc_layer(x_tensor, num_outputs):

    fc = tf.contrib.layers.fully_connected(x_tensor,
                                           num_outputs,
                                           activation_fn=tf.nn.sigmoid,
                                           weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                           # weights_initializer=tf.contrib.layers.xavier_initializer(dtype=tf.float64),
                                           biases_initializer=tf.zeros_initializer())

    fc_d = tf.nn.dropout(fc, keep_prob)

    return fc_d

fc1 = fc_layer(inputs, n_unit[0])
fc2 = fc_layer(fc1, n_unit[1])
fc3 = fc_layer(fc2, n_unit[2])
fc4 = fc_layer(fc3, n_unit[3])
fc5 = fc_layer(fc4, n_unit[4])
logits = tf.contrib.layers.fully_connected(fc5,
                                           1,
                                           activation_fn=None,
                                           weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                           # weights_initializer=tf.contrib.layers.xavier_initializer(dtype=tf.float64),
                                           biases_initializer=tf.zeros_initializer())

In [ ]:
prob = tf.squeeze(logits)

weights = weights / tf.reduce_sum(weights)

# cost = - tf.reduce_sum(weights * (labels * tf.log(prob) +
#                         (tf.ones_like(labels, dtype=tf.float64) - labels) * tf.log(tf.ones_like(labels, dtype=tf.float64)-prob)))

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
def era_k_fold_with_weight(x, y, w, e, n_valid, n_cv):

    for i in range(n_cv):

        era_idx = list(range(1, 21))
        valid_group = np.random.choice(era_idx, n_valid, replace=False)

        train_index = []
        valid_index = []

        for ii, ele in enumerate(e):

            if ele in valid_group:
                valid_index.append(ii)
            else:
                train_index.append(ii)

        np.random.shuffle(train_index)
        np.random.shuffle(valid_index)

        # Training data
        x_train = x[train_index]
        y_train = y[train_index]
        w_train = w[train_index]

        # Validation data
        x_valid = x[valid_index]
        y_valid = y[valid_index]
        w_valid = w[valid_index]

        yield x_train, y_train, w_train, x_valid, y_valid, w_valid

In [ ]:
def get_batches(x, y, w, batch_num):

    n_batches = len(x) // batch_num

    for ii in range(0, n_batches * batch_num, batch_num):

        if ii != n_batches * batch_num:
            batch_x, batch_y, batch_w = x[ii: ii + batch_num], y[ii: ii + batch_num], w[ii: ii + batch_num]

        else:
            batch_x, batch_y, batch_w = x[ii:], y[ii:], w[ii:]

        yield batch_x, batch_y, batch_w
        

with tf.Session() as sess:

    start_time = time.time()

    cv_counter = 0

    prob_total = []

    for x_train, y_train, w_train, \
        x_valid, y_valid, w_valid in era_k_fold_with_weight(x_train_d,
                                                            y_train_d,
                                                            w_train_d,
                                                            e_train_d,
                                                            4, 20):

        cv_counter += 1

        print('======================================================================================================')
        print('Training on the Cross Validation Set: {}'.format(cv_counter))
        
        sess.run(tf.global_variables_initializer())

        batch_counter = 0

        for epoch_i in range(epochs):

            for batch_i, (batch_x, batch_y, batch_w) in enumerate(get_batches(x=x_train,
                                                                              y=y_train,
                                                                              w=w_train,
                                                                              batch_num = batch_size)):
                

                batch_counter += 1

                _, logits_1 = sess.run([optimizer, logits],
                                   {inputs: batch_x,
                                    labels: batch_y,
                                    weights: batch_w,
                                    keep_prob: keep_prob_})
                
                print(logits_1)

                if batch_counter % display_step == 0 and batch_i > 0:
                    
                    print(batch_x, batch_y, batch_w)

                    cost_train = sess.run(cost,{inputs: batch_x,
                                                labels: batch_y,
                                                weights: batch_w,
                                                keep_prob: 1.0})
                    print(cost_train)

                    cost_valid_a = []

                    for iii, (valid_batch_x,
                              valid_batch_y,
                              valid_batch_w) in enumerate(get_batches(x_valid,
                                                                      y_valid,
                                                                      w_valid,
                                                                      batch_size)):

                        cost_valid_i = sess.run(cost,{inputs: valid_batch_x,
                                                      labels: valid_batch_y,
                                                      weights: valid_batch_w,
                                                      keep_prob: 1.0})

                        cost_valid_a.append(cost_valid_i)

                    cost_valid = sum(cost_valid_a) / len(cost_valid_a)

                    total_time = time.time() - start_time

                    print('CV: {} |'.format(cv_counter),
                          'Epoch: {}/{} |'.format(epoch_i + 1, epochs),
                          'Batch: {} |'.format(batch_counter),
                          'Time: {:>3.2f}s |'.format(total_time),
                          'Train_Loss: {:>.8f} |'.format(cost_train),
                          'Valid_Loss: {:>.8f}'.format(cost_valid))

                    print(sess.run(logits, {inputs: x_test_d, keep_prob: 1.0, is_train: False}))

        # Prediction
        print('Predicting...')

        logits_ = sess.run(logits, {inputs: x_test,
                                    keep_prob: 1.0,
                                    is_train: False})

        logits_ = logits_.flatten()

        prob_test = 1.0 / (1.0 + np.exp(-logits_))

        prob_total.append(list(prob_test))

        utils.save_pred_to_csv(pred_path + 'dnn_cv_{}_'.format(cv_counter), id_test, prob_test)

    # Final Result
    print('======================================================================================================')
    print('Calculating final result...')

    prob_mean = np.mean(np.array(prob_total), axis=0)

    utils.save_pred_to_csv(pred_path + 'dnn_', id_test, prob_mean)

In [ ]:
a = [[0.8, 0.2], 
     [0.3, 0.7]]

In [ ]:
np.array(a)[:, 1]